In [ ]:
import yadisk
import http.server
import json
import urllib.parse
import urllib.request
from typing import List

# Настройки
YANDEX_DISK_API = "https://cloud-api.yandex.net/v1/disk/resources"
OAUTH_TOKEN = "y0__xC3gtebARjTwTwgmaKj4xXkkA33pgz-WKrYhwZS2PTMluDXtw"
FILES_PATH = "/app"
PORT = 8000

def get_disk_files() -> List[str]:
    # Получает список имён файлов из указанной папки на Яндекс Диске
    # Реализует пагинацию для обработки более 1000 файлов
    files = []
    params = {
        "path": FILES_PATH,
        "limit": 1000,
        "offset": 0
    }

    while True:
        url = f"{YANDEX_DISK_API}?{urllib.parse.urlencode(params)}"
        request = urllib.request.Request(url)
        request.add_header("Authorization", f"OAuth {OAUTH_TOKEN}")

        try:
            with urllib.request.urlopen(request) as response:
                data = json.loads(response.read().decode("utf-8"))

                for item in data.get("_embedded", {}).get("items", []):
                    if item["type"] == "file":
                        files.append(item["name"])

                total = data["_embedded"]["total"]
                limit = data["_embedded"]["limit"]
                offset = data["_embedded"]["offset"]
                if offset + limit >= total:
                    break
                params["offset"] += limit

        except Exception as e:
            print(f"Ошибка при запросе к API: {e}")
            break

    return files

def generate_html(uploaded_files: List[str]) -> str:
    # Формирует HTML‑страницу со списком файлов
    # Подсвечивает зелёным файлы, присутствующие на Диске
    all_files = [
        "document.txt",
        "image.jpg",
        "script.py",
        "data.csv",
        "notes.md"
    ]

    html = f"""
    <html>
    <head>
        <meta charset="utf-8">
        <title>Файлы на Яндекс Диске</title>
        <style>
            body {{
                font-family: Arial, sans-serif;
                margin: 20px;
                line-height: 1.6;
            }}
            h1 {{
                color: #333;
            }}
            ul {{
                list-style-type: none;
                padding: 0;
            }}
            li {{
                padding: 12px;
                margin: 5px 0;
                border: 1px solid #ddd;
                border-radius: 4px;
            }}
            .uploaded {{
                background-color: rgba(0, 200, 0, 0.25);
                font-weight: bold;
            }}
        </style>
    </head>
    <body>
        <h1>Файлы в папке: {FILES_PATH}</h1>
        <ul>
    """

    for file in all_files:
        if file in uploaded_files:
            html += f'<li class="uploaded">{file}</li>'
        else:
            html += f'<li>{file}</li>'

    html += """
        </ul>
        <p><small>Подсвечены файлы, которые уже есть на Яндекс Диске.</small></p>
    </body>
    </html>
    """
    return html

class RequestHandler(http.server.BaseHTTPRequestHandler):
    def do_GET(self):
        uploaded_files = get_disk_files()
        html_content = generate_html(uploaded_files)
        self.send_response(200)
        self.send_header("Content-type", "text/html; charset=utf-8")
        self.end_headers()
        self.wfile.write(html_content.encode("utf-8"))

def run_server():
    # Запускает HTTP‑сервер на указанном порту
    # Выводит в консоль информацию о запуске и инструкции по остановке
    server = http.server.HTTPServer(("", PORT), RequestHandler)
    print(f"Сервер запущен: http://localhost:{PORT}")
    print("Для остановки нажмите Ctrl+C")
    server.serve_forever()

if __name__ == "__main__":
    run_server()
